## 0. Importing all necessary libs and tools

In [ ]:
import torch
from ultralytics import YOLO
import os
import time
import sys
for module_name in list(sys.modules.keys()):
    if module_name.startswith('src.'):
        del sys.modules[module_name]
from src.config import MODEL_NAME, OVERLAPPING_PERCENTAGE, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD
print(MODEL_NAME, SLICE_SIZE, SLICES_FOLDER, PREDICT_FOLDER_PREFIX, CONFIDENCE_THRESHOLD, IOU_FOLDER_PREFIX, OUTLIER_FILTER_FOLDER_PREFIX, OUTLIER_THRESHOLD_K, IOU_THRESHOLD)
from src.model import get_devices, get_models, process_folder, read_annotation_file, create_annotation_file, process_folders_parallel
from src.tools.utils import get_destination_folder, create_destination_folder, get_slice_coordinates
from src.tools.image_slicer import create_image_slices


best_4 640 slices predicted_images_with_annotations 0.4 iou_filtered outlier_filtered 3 0.5


## 1. Slicing image process


In [ ]:
images_folder = "dataset/0.0.1/АФС для обработки ИИ"
image_paths = [f"{images_folder}/{image_path}" for image_path in os.listdir(images_folder)]
for image_path in image_paths:
    create_image_slices(
        image_path=image_path, 
        overlap_percentage=OVERLAPPING_PERCENTAGE, 
        destination_folder=SLICES_FOLDER,
        slice_size=SLICE_SIZE
    )

## 1.2 Parallel slicing approach

In [ ]:
from src.tools.image_slicer import create_images_slices_parallel
create_images_slices_parallel()

Processing 211 images using 20 processes


Processing images: 100%|██████████| 211/211 [00:59<00:00,  3.55it/s]


Completed: 211 images, 37136 total slices


## 2. Processing slices with yolo model 

In [ ]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
print(f"Using devices: {devices}")
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD, "2"])
output_folder = create_destination_folder(folder_name)
boxes_list = process_folders(models[1], devices[1], output_folder)

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']
Модель загружена на: cuda:0
Модель загружена на: cuda:1
Using devices: ['cuda:0', 'cuda:1']
Обработка изображений на устройстве: cuda:1
пПапка вывода: predicted_images_with_annotations-best_4-0.4-2
Обработано: 10 изображений с детекциями
Обработано: 20 изображений с детекциями
Обработано: 30 изображений с детекциями
Обработано: 40 изображений с детекциями
Обработано: 50 изображений с детекциями
Обработано: 60 изображений с детекциями
Обработано: 70 изображений с детекциями
Обработано: 80 изображений с детекциями
Обработано: 90 изображений с детекциями
Обработано: 100 изображений с детекциями
Обработано: 110 изображений с детекциями
Обработано: 120 изображений с детекциями


KeyboardInterrupt: 

## 2.2 Processing folders parallel

In [ ]:
devices = get_devices()
models = get_models(MODEL_NAME, devices)
folder_name = get_destination_folder([PREDICT_FOLDER_PREFIX, MODEL_NAME, CONFIDENCE_THRESHOLD, "2"])
output_folder = create_destination_folder(folder_name)
total_detections = process_folders_parallel(models, devices, output_folder)

CUDA доступна: True
Название GPU: NVIDIA GeForce RTX 5090
Количество GPU: 2
Текущий GPU device: 0
Доступные устройства: ['cuda:0', 'cuda:1']
Модель загружена на: cuda:0
Модель загружена на: cuda:1


Processing folders:   0%|          | 0/211 [00:00<?, ?it/s]